In [1]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

In [94]:
import warnings 
warnings.filterwarnings('ignore')
lookback_window = [0, 50]
number_of_clusters = 38

def cluster_composition_and_centroid(self):
## cluster composition and centroids

result = dict(zip(list(self.correlation_matrix.columns), self.apply_SPONGE())) ## composition

df_cleaned['Cluster'] = df_cleaned.index.map(result)
centroid_returns = df_cleaned.groupby('Cluster').mean() ## centroids 

df_cleaned = df_cleaned.transpose()
centroid_returns = centroid_returns.transpose()

## constituent_weights
centred_returns = df_cleaned.copy()
sigma = 0.01

constituent_weights = pd.DataFrame(index=['Weight'], columns=centred_returns.columns)
total_weight = pd.DataFrame(index=['Total weight'], columns=[i for i in range(number_of_clusters)], data=np.zeros((1, number_of_clusters)))

## we first compute the difference between the cluster centroid return and the cluster ticker return
for ticker in centred_returns.columns:
    centred_returns[ticker][:-1] = centred_returns[ticker][:-1] - centroid_returns[int(centred_returns[ticker]['Cluster'])]

## we use this difference to compute the distance between each asset and its cluster centroid return 
for ticker in centred_returns.columns:
    constituent_weights[ticker] = np.exp(sigma*((np.linalg.norm(centred_returns[ticker][:-1]))**2)/2)
    total_weight[int(centred_returns[ticker]['Cluster'])]['Total weight'] += np.exp(sigma*((np.linalg.norm(centred_returns[ticker][:-1]))**2)/2)

## we normalize the weights
for ticker in centred_returns.columns:
    constituent_weights[ticker] = constituent_weights[ticker]['Weight']/total_weight[int(centred_returns[ticker]['Cluster'])]['Total weight']

## check whether the weights equal to 0 within each cluster: 
# constituent_weights[[ticker for ticker in centred_returns.columns if centred_returns[ticker]['Cluster']  == 1.0]].sum(axis=1)
    
## cluster returns 
    
cluster_returns = pd.DataFrame(index=df_cleaned.index[:-1], columns=np.arange(number_of_clusters), data=np.zeros((df_cleaned.shape[0] - 1, number_of_clusters)))

for ticker in df_cleaned.columns:
    cluster_returns[int(df_cleaned[ticker]['Cluster'])] = cluster_returns[int(df_cleaned[ticker]['Cluster'])] + constituent_weights[ticker]['Weight'] * df_cleaned[ticker][:-1]

IndentationError: expected an indented block after function definition on line 6 (2352418720.py, line 9)

In [2]:
from pypfopt.efficient_frontier import EfficientFrontier

from PyFolioCC import PyFolioCC 

historical_data = df_cleaned
lookback_window = [0, 50]
evaluation_window = 1
number_of_clusters = 38
clustering_method = 'SPONGE'
sigma = 0.01

portfolio = PyFolioCC(historical_data=historical_data, lookback_window=lookback_window, evaluation_window=evaluation_window, number_of_clusters=number_of_clusters, sigma=sigma)


/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/PyFolioCC.py:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  A_pos, A_neg = self.correlation_matrix.applymap(lambda x: x if x >= 0 else 0), self.correlation_matrix.applymap(lambda x: abs(x) if x < 0 else 0)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Code/PyFolioCC.py:309: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, becaus

In [13]:
returns = portfolio.cluster_returns
returns


,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
03/01/2000,-0.000082,0.000165,-0.000003,-0.000015,-1.640796e-04,2.414832e-06,-0.000002,-0.000006,-0.000002,-0.000015,...,-0.000146,-9.666162e-07,-0.000006,0.0,-0.000171,0.000000e+00,0.000043,-0.000005,0.000002,-8.673617e-19
04/01/2000,-0.000239,0.000123,0.000012,0.000016,-1.018894e-04,-4.242877e-06,0.000004,-0.000051,0.000005,-0.000045,...,-0.000073,8.799520e-06,0.000004,0.0,-0.000045,0.000000e+00,-0.000014,-0.000047,0.000082,0.000000e+00
05/01/2000,-0.000226,-0.000143,0.000011,-0.000011,1.493832e-04,1.228760e-05,0.000013,-0.000028,0.000083,0.000011,...,0.000098,-4.725416e-07,-0.000015,0.0,0.000046,0.000000e+00,0.000071,0.000006,0.000003,0.000000e+00
06/01/2000,0.000035,-0.000050,-0.000003,-0.000006,2.643111e-05,-5.899025e-06,-0.000022,-0.000053,0.000010,-0.000047,...,0.000098,4.340347e-06,-0.000041,0.0,-0.000047,-8.673617e-19,-0.000048,-0.000002,0.000029,0.000000e+00
07/01/2000,-0.000015,-0.000016,-0.000036,-0.000015,-2.662359e-05,1.194595e-05,-0.000015,0.000022,0.000025,0.000008,...,-0.000051,-5.841073e-06,0.000008,0.0,0.000075,0.000000e+00,-0.000039,0.000043,0.000013,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24/12/2020,0.000021,-0.000069,0.000004,-0.000030,-7.717685e-08,9.045304e-07,-0.000005,-0.000070,-0.000002,-0.000010,...,-0.000094,-8.675542e-06,0.000023,0.0,0.000026,-2.168404e-19,-0.000018,-0.000003,-0.000036,0.000000e+00
28/12/2020,0.000194,0.000073,0.000060,-0.000029,2.137172e-05,-1.177202e-05,-0.000009,-0.000087,0.000018,-0.000023,...,-0.000011,-9.255886e-07,-0.000004,0.0,-0.000018,0.000000e+00,-0.000027,0.000014,-0.000027,1.734723e-18
29/12/2020,-0.000115,-0.000005,0.000022,-0.000047,1.190333e-05,-7.891361e-06,-0.000003,-0.000041,0.000006,-0.000045,...,0.000046,-1.019297e-05,-0.000011,0.0,-0.000012,1.734723e-18,-0.000009,-0.000031,-0.000065,0.000000e+00
30/12/2020,-0.000070,0.000079,-0.000006,0.000018,1.079262e-04,5.814421e-06,0.000012,-0.000050,-0.000074,0.000040,...,-0.000024,4.241239e-06,0.000023,0.0,0.000123,0.000000e+00,0.000035,0.000011,0.000023,0.000000e+00
